# Medallion Architecture — Bronze Layer

Ingests the raw TPC-H tables into a `retail_bronze` schema with audit columns
(`_ingested_at`, `_source_table`, `_batch_id`), no business transformations applied.

Liquid clustering is added on large tables for downstream read performance.

**Prereq**: Run `00_generate_data.ipynb` first.

## 1 — Configuration

In [ ]:
from pyspark.sql import functions as F
from datetime import datetime

# ── Config ─────────────────────────────────────────────────────────────────────
CATALOG       = spark.catalog.currentCatalog()
SOURCE_SCHEMA = "tpch"           # where raw TPC-H tables live
BRONZE_SCHEMA = "retail_bronze"  # target bronze schema

source_fqn = f"{CATALOG}.{SOURCE_SCHEMA}"
bronze_fqn = f"{CATALOG}.{BRONZE_SCHEMA}"

# Batch metadata
BATCH_ID     = datetime.now().strftime("%Y%m%d_%H%M%S")
INGESTED_AT  = F.current_timestamp()

print(f"Catalog       : {CATALOG}")
print(f"Source schema : {source_fqn}")
print(f"Bronze schema : {bronze_fqn}")
print(f"Batch ID      : {BATCH_ID}")

## 2 — Create Bronze Schema

In [ ]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {bronze_fqn}")
spark.sql(f"COMMENT ON SCHEMA {bronze_fqn} IS 'Bronze layer — raw ingestion of TPC-H retail data with audit columns'")
print(f"Schema ready: {bronze_fqn}")

## 3 — Generic Bronze Ingestion Function

Every Bronze table gets three audit columns appended:
| Column | Purpose |
|--------|---------|
| `_ingested_at` | Timestamp of when this row was loaded into Bronze |
| `_source_table` | Fully qualified name of the upstream source |
| `_batch_id` | Identifies this particular load run |

In [ ]:
def ingest_to_bronze(table_name, cluster_cols=None):
    """
    Read from source, stamp audit columns, write to Bronze as managed Delta.
    Optionally applies liquid clustering for large tables.
    """
    src = f"{source_fqn}.{table_name}"
    tgt = f"{bronze_fqn}.{table_name}"

    df = (
        spark.table(src)
        .withColumn("_ingested_at",  INGESTED_AT)
        .withColumn("_source_table", F.lit(src))
        .withColumn("_batch_id",     F.lit(BATCH_ID))
    )

    # Write as managed Delta table
    df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(tgt)

    # Apply liquid clustering on large tables for query performance
    if cluster_cols:
        cols = ", ".join(cluster_cols)
        spark.sql(f"ALTER TABLE {tgt} CLUSTER BY ({cols})")

    cnt = spark.table(tgt).count()
    print(f"  ✓ {tgt:<50} {cnt:>12,} rows  (clustered by: {cluster_cols or 'none'})")
    return cnt

## 4 — Ingest All Tables to Bronze

In [ ]:
# Table → optional liquid clustering columns (pick high-cardinality filter/join keys)
bronze_tables = {
    "region":   None,
    "nation":   None,
    "supplier": ["s_nationkey"],
    "part":     ["p_brand", "p_type"],
    "partsupp": ["ps_partkey", "ps_suppkey"],
    "customer": ["c_nationkey", "c_mktsegment"],
    "orders":   ["o_orderdate", "o_custkey"],
    "lineitem": ["l_shipdate", "l_orderkey"],
}

print(f"{'Table':<50} {'Rows':>12}  Clustering")
print("=" * 85)

total = 0
for tbl, cols in bronze_tables.items():
    total += ingest_to_bronze(tbl, cluster_cols=cols)

print("=" * 85)
print(f"  Total rows in Bronze: {total:,}")

## 5 — Add Table-Level Comments (Documentation)

In [ ]:
table_comments = {
    "region":   "Reference table: 5 world regions (TPC-H R_).",
    "nation":   "Reference table: 25 nations mapped to regions (TPC-H N_).",
    "supplier": "Dimension: suppliers with contact info and nation (TPC-H S_).",
    "part":     "Dimension: product catalog with brand, type, size (TPC-H P_).",
    "partsupp": "Bridge: part-supplier availability and cost (TPC-H PS_).",
    "customer": "Dimension: customers with segment and nation (TPC-H C_).",
    "orders":   "Fact: order headers with status, priority, dates (TPC-H O_).",
    "lineitem": "Fact: order line items — largest table, grain of all analytics (TPC-H L_).",
}

for tbl, comment in table_comments.items():
    spark.sql(f"COMMENT ON TABLE {bronze_fqn}.{tbl} IS '{comment}'")

print("Table comments applied.")

## 6 — Validate: Row Counts & Schema Snapshot

In [ ]:
# Quick validation: compare source vs bronze row counts
from pyspark.sql import Row

validation = []
for tbl in bronze_tables:
    src_cnt = spark.table(f"{source_fqn}.{tbl}").count()
    brz_cnt = spark.table(f"{bronze_fqn}.{tbl}").count()
    match   = "✓" if src_cnt == brz_cnt else "✗ MISMATCH"
    validation.append(Row(table=tbl, source_rows=src_cnt, bronze_rows=brz_cnt, status=match))

df_val = spark.createDataFrame(validation)
display(df_val)

## 7 — Demonstrate Delta Features: History & Time Travel

In [ ]:
# Delta table history — shows every operation, useful for audit
display(spark.sql(f"DESCRIBE HISTORY {bronze_fqn}.lineitem"))

In [ ]:
# Delta detail — file count, size, clustering info
display(spark.sql(f"DESCRIBE DETAIL {bronze_fqn}.lineitem"))

---
All 8 tables are in `retail_bronze` with audit columns and liquid clustering.

Continue with `02_silver_layer.ipynb`.